In [2]:
import csv
import os
import pandas as pd

## 1. Generación de archivo binario UpSet

In [3]:
def generate_UpSetFile(Lists,output_filename,threshold):
    """
    Crea un archivo que UpSet-codifica el contenido de términos funcionales en distintas listas de genes D.E.
    
    Debe recibir una lista con el nombre de los archivos que resultan de las query en g:Profiler para cada lista de genes D.E. 
    Tales archivos son csv con el siguiente encabezado:
    
    [source - term_name - term_id - adjusted_p_value - negative_log10_of_adjusted_p_value - term_size - query_size - intersection_size - effective_domain_size - intersections]     
    
    Parámetros:
    -----------
        Lists [string]: arreglo con los nombres de las listas de términos funcionales. Por ejemplo: ['Car_up','Car_down',...]
        output_filename (string): nombre del archivo a generar. Por ejemplo: 'UpSetTerminosFuncionales.txt'
    
    """
    UpSetDict = {}
    
    for listID in Lists:
        col = {} #Aquí se guardará la relación entre el título de una columna y su número de columna
        row_count = 0
        with open(listID + '_CSV.txt') as actual_list:
            csv_reader = csv.reader(actual_list, delimiter=',')
            for row in csv_reader:
                
                if row_count == 0: #Encabezado
                    for i in range(0,len(row)): #Guardamos la relación entre el título de una columna y su número de columna
                        col[row[i]]=i
                        
                else: #Entradas
                    termID = row[col['term_id']]
                    if not termID in UpSetDict: #(Si el término termID no está como llave en el diccionario)
                        
                        #Inicializaciones
                        UpSetDict[termID]={}
                        UpSetDict[termID]['term_name']=row[col['term_name']]
                        UpSetDict[termID]['term_source']=row[col['source']]
                        for listID_aux in Lists: 
                            UpSetDict[termID][listID_aux] = 0 #inicializamos todas las pertenencias en 0
                            UpSetDict[termID][listID_aux + '_adj_pvalue'] = 1 #inicializamos los adjusted pvalue en 1
                            
                        #Determinacion de pertenencia y adj-pvalue
                        if float(row[col['adjusted_p_value']])<threshold: #restriccion sobre el adjusted p-value para que la lista invoque al termino
                            UpSetDict[termID][listID] = 1
                        UpSetDict[termID][listID + '_adj_pvalue'] = row[col['adjusted_p_value']]
                        
                    else: #(Si el termino termID ya fue agregado)
                        if float(row[col['adjusted_p_value']])<threshold:
                            UpSetDict[termID][listID] = 1
                        UpSetDict[termID][listID + '_adj_pvalue'] = row[col['adjusted_p_value']]
                row_count+=1
        
        with open(output_filename,'w') as f:
            csv_writer = csv.writer(f,delimiter='\t')
            
            #Escribir el encabezado
            header_line = ['term_id','term_name','term_source']
            for listID in Lists:
                header_line.append(listID)
            for listID in Lists:
                header_line.append(listID+'_adj_pvalue')
            csv_writer.writerow(header_line)
            
            for termID in UpSetDict:
                new_line = [termID]
                for field in header_line:
                    if field!='term_id':
                        new_line.append(UpSetDict[termID][field])
                csv_writer.writerow(new_line)

In [6]:
listIDs = ['He_down','He_up','Lind_down','Lind_up','Car(Aae)_down','Car(Aae)_up','Bai_F_abd_down','Bai_F_abd_up','Bai_F_head_down','Bai_F_head_up','Bai_M_abd_down','Bai_M_abd_up','Bai_M_head_down','Bai_M_head_up']
output_filename = 'Terms_UpSetFile.txt'
generate_UpSetFile(listIDs,output_filename,0.25)